# Basic Steps to do Structuring

* The script "make_db.py" loads data from S3 using Spark, and parallel inserts the data into a RDS and PostgresSQL - including making table
* Run "spark submit make_db.py"
* Read raw data from S3
* Create Spark dataFrames in 3 nf


* RDS - PostgreSQL

* Read data into each table using a spark script


###### Please note, I am trying to run the code in Notebook here to make it look modular. The reason I am running into errors here is I have put some of the functions and other files in separate folders for proper book keeping. So from this location I would have to change the paths of the functions I am calling. Recommended way to run this--- All the scripts will run in PyCharm or Atom. 

In [3]:
from os.path import expanduser
import json
import yaml
import psycopg2
from pyspark import SparkContext

# We run this on EMR. Make sure to install psycopg2

global my_cred


def spark_job(sc, bucket_path):
    rdd = sc.textFile(bucket_path)
    print("Distributing jobs to partitions")
    rdd.foreachPartition(partition_job)
    print("All partition jobs completed. Exiting...")


def partition_job(source, credentials=None):

    if not credentials:
        credentials = api_cred.value

    conn, cur = connect_to_psql(credentials)

    query = """INSERT INTO reviews (artist, album, score, album_art, genre, label,
                pub_date, abstract, review_content)
                VALUES (%(artist)s, %(album)s, %(score)s, %(album_art)s, %(genre)s,
                %(label)s, %(pub_date)s, %(abstract)s, %(review_content)s)"""

    print("Starting job...")

    for item in source:

        try:
            if type(item) == unicode:
                data = json.loads(item)
            elif type(item) == dict:
                data = item
                print('Inserting album: ', data['album'])
            cur.execute(query, data)
        except:
            conn.rollback()

        conn.commit()

    print("Partition job completed.")

def make_table():

    conn, cur = connect_to_psql()

    query = """CREATE TABLE IF NOT EXISTs reviews
                (id SERIAL PRIMARY KEY, artist varchar(255), album varchar(255),
                 score varchar(255), album_art varchar(255), genre varchar(255),
                 label varchar(255), pub_date TIMESTAMP, abstract TEXT,
                 featured_tracks varchar(255), review_content TEXT)"""
    try:
        cur.execute(query)
    except:
        print("Something bad happenned while making tables...?")
    
    conn.commit()
    print("Finished creating reviews table.")


def connect_to_psql(credentials=None):

    if not credentials:
        credentials = my_cred.value

    try:
        print("Connecting...")
        conn = psycopg2.connect(**credentials['rds'])
        cur = conn.cursor()
        print("Connection Established")
    except:
        print("Error Establishing Connection (bad credentials?)")
        return False

    return conn, cur

ModuleNotFoundError: No module named 'pyspark'

In [7]:
def connect_to_psql(credentials=None):

	if not credentials:
		credentials = my_cred.value

	try:
		print("Connecting...")
		conn = psycopg2.connect(**credentials['rds'])
		cur = conn.cursor()
		print("Connection Established")
	except:
		print("Error Establishing Connection (bad credentials?)")
		return False

	return conn, cur


my_cred = yaml.load(open(expanduser('~/Desktop/api_cred.yml')))
conn, cur = connect_to_psql(my_cred)

Connecting...
Connection Established


# Creating table in 3NF form

###### The RDS database from which we query and get the data below, is basically the columns of data on artist name, genre, album, etc

query = """CREATE TABLE IF NOT EXISTs reviews
                (id SERIAL PRIMARY KEY, artist varchar(255), album varchar(255),
                 score varchar(255), album_art varchar(255), genre varchar(255),
                 label varchar(255), pub_date TIMESTAMP, abstract TEXT,
                 featured_tracks varchar(255), review_content TEXT)"""
                 
Since it is reviews we are parsing the table we obtain itself is in 3NF, so it does NOT serve anything extra useful to create multipe tables.

We are extracting exactly the information we want using the script, as well as the query below.

In this case the Primary key (PK) would be "artist".

In [14]:
cur.execute("""SELECT album, artist, pub_date FROM reviews LIMIT 10;""")

In [15]:
for item in cur:
    print(item)

('Jamie, My Intentions Are Bass EP', '!!!', datetime.datetime(2010, 11, 1, 6, 0))
('Konstellaatio', '', datetime.datetime(2014, 2, 27, 6, 0, 4))
('As If', '!!!', datetime.datetime(2015, 10, 21, 6, 0))
('New Heaven', '1,2,3', datetime.datetime(2011, 6, 21, 6, 0, 3))
('11 11', '11 11', datetime.datetime(2014, 7, 29, 6, 0, 1))
('120 Days II', '120 Days', datetime.datetime(2012, 4, 13, 6, 0, 4))
('2 54', '2 54', datetime.datetime(2012, 6, 15, 6, 0, 3))
('The Other I', '2 54', datetime.datetime(2014, 11, 11, 6, 0, 3))
('Own Your Ghost', '13 & God', datetime.datetime(2011, 5, 17, 6, 0))
('B.O.A.T.S II  Me Time', '2 Chainz', datetime.datetime(2013, 9, 18, 6, 0, 1))
